# J1: JSSP Basic (Static) - Multiagent Pattern


<img src="../img/multi_agent_pattern.png" alt="Alt text" width="500"/>

We follow the concept of crewai with our implementation. We create a crew which is a structured group of AI agents that collaborate to complete complex tasks together.

Each agent has a distinct role. We declare dependencies between agents. This way the output of one agent, can be the input for another agent.

**J1: JSSP Basic (Static)** - Basic job-shop scheduling with fixed machine assignments.


In [ ]:
#%pip install -r requirements.txt
%pip install --upgrade pip
%pip install jupyter ipykernel
%%python3 -m ipykernel install --user --name=python310 --display-name "Python 3.10"


In [ ]:
import sys
print("Python version:", sys.version)


In [ ]:
%pip install openai python-dotenv graphviz colorama


In [ ]:
import openai
import dotenv
import graphviz
import colorama
print("✅ All dependencies installed successfully!")


In [ ]:
import sys
import os

project_root = os.getcwd()
src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

print("Updated sys.path:", sys.path)


## The Agent Class


First of all, we need an **Agent Class**. This class implements an Agent, and internally it implements the ReAct technique.


In [ ]:
import sys
import os
os.environ["OPENAI_API_KEY"]=""
print("API Key set:", os.getenv("OPENAI_API_KEY") is not None)


In [ ]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.multi_agent.agent import Agent
print("✅ Agent imported successfully!")


You can also associate tools with the agent. Let's create a tool for writing some string into a file.


In [ ]:
from src.tool_agent.tool import tool

@tool
def write_str_to_txt(string_data: str, txt_filename: str):
    """
    Writes a string to a txt file.

    This function takes a string and writes it to a text file. If the file already exists, 
    it will be overwritten with the new data.

    Args:
        string_data (str): The string containing the data to be written to the file.
        txt_filename (str): The name of the text file to which the data should be written.
    """
    # Write the string data to the text file
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")


## The Crew for J1: JSSP Basic (Static)


In [ ]:
from src.multi_agent.crew import Crew

with Crew() as crew:
    # ---- Job & Machine Setup Agent ---- #
    JM_Agent = Agent(
        name="Job & Machine Setup Agent",
        backstory="You define jobs, machines, and processing times for the job shop scheduling problem.",
        task_description="Set up JSSP instance: define jobs with their operations, machines available, and processing times for each job-operation-machine combination.",
        task_expected_output="Structured JSSP data: 1) Jobs with operation sequences. 2) Machines available. 3) Processing times matrix (job × operation × machine)."
    )
    
    # ---- Schedule Generation Agent ---- #
    SG_Agent = Agent(
        name="Schedule Generation Agent",
        backstory="You generate a feasible schedule for all jobs on machines respecting operation sequences.",
        task_description="Create a schedule assigning each operation to a machine at a specific time, respecting: job operation sequences, machine capacity (one job at a time), and no preemption.",
        task_expected_output="Feasible schedule with start/end times for each operation, machine assignments, and job completion times."
    )
    
    # ---- Makespan Optimization Agent ---- #
    MO_Agent = Agent(
        name="Makespan Optimization Agent",
        backstory="You optimize the schedule to minimize makespan (total completion time).",
        task_description="Optimize the schedule to minimize makespan while maintaining feasibility, using scheduling heuristics and optimization techniques.",
        task_expected_output="Optimized schedule with improved makespan, operation sequences, and machine utilization."
    )
    
    # ---- Constraint Validation Agent ---- #
    CV_Agent = Agent(
        name="Constraint Validation Agent",
        backstory="You validate that the schedule satisfies all JSSP constraints.",
        task_description="Validate schedule: check operation sequences are respected, no machine conflicts (one job at a time), and all operations are scheduled.",
        task_expected_output="Validated schedule with constraint satisfaction report confirming all constraints are met."
    )

    Writer_agent = Agent(
        name="Writer Agent",
        backstory="You are a language model specialised in writing text into .json files",
        task_description="Write the json response into './j1_output.json'",
        task_expected_output="A .json file containing the given string",
        tools=write_str_to_txt,
    )
    
    # ---- Define Dependencies ---- #
    JM_Agent >> SG_Agent >> MO_Agent >> CV_Agent >> Writer_agent


In [ ]:
crew.run()
